In [42]:
WRKDIR = ''
DOWNLOADDIR = '{}/pipelinedownloads'.format(WRKDIR)
GCOUTPUT = ''
USER = ''
PROJECT_ID = ''
BUCKET = ''

## Download pipeline python scripts from github

In [31]:
!curl https://raw.githubusercontent.com/nerettilab/RepEnrich2/master/RepEnrich2.py \
    -o {DOWNLOADDIR}/RepEnrich2.py
!curl https://raw.githubusercontent.com/nerettilab/RepEnrich2/master/RepEnrich2_setup.py \
    -o {DOWNLOADDIR}/RepEnrich2_setup.py
!curl https://raw.githubusercontent.com/nerettilab/RepEnrich2/master/RepEnrich2_subset.py \
    -o {DOWNLOADDIR}/RepEnrich2_subset.py



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19849  100 19849    0     0    98k      0 --:--:-- --:--:-- --:--:--   98k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9665  100  9665    0     0  68063      0 --:--:-- --:--:-- --:--:-- 68063
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6058  100  6058    0     0  43582      0 --:--:-- --:--:-- --:--:-- 43582


## Push scripts to google cloud 
#fill in path to where you coped RepEnrich scripts for the gsutil

In [ ]:
%%bash -s "$DOWNLOADDIR"
DOWNLOADDIR=${1}
cd ${DOWNLOADDIR}
##fill in path to where you coped RepEnrich scripts
gsutil cp RepEnrich2_setup.py gs://
gsutil cp RepEnrich2_subset.py gs://
gsutil cp RepEnrich2.py gs://

# Step 1) Attain repetitive element annotation
## Use (a) the provided cleaned file or (b) the full repeatmasker file

### (a) Cleaned repeatmasker file 
download `hg38_repeatmasker_clean.txt.gz` and `Repenrich2_setup_hg38.tar.gz` from [here](https://drive.google.com/drive/folders/0B8_2gE04f4QWNmdpWlhaWEYwaHM)

In [ ]:
%%bash -s "$DOWNLOADDIR"
DIR=${1}
cd ${DIR}
gunzip hg38_repeatmasker_clean.txt.gz

In [ ]:
print("Run at terminal (already done):")
print("gsutil cp \
{}/hg38_repeatmasker_clean.txt \
{}/resourcestest/".format(DOWNLOADDIR,BUCKET))

you can use the provided repetitive element indexed files (`RepEnrich2_setup_hg38`) OR generate them using the setup script on the `hg38_repeatmasker_clean.txt` annotation file (go to Step 2)

In [ ]:
print("run this to copy the indexed files to google cloud if you aren't generating them yourself (already done)")
print("gsutil cp \
{}/Repenrich2_setup_hg38.tar.gz \
{}/resourcestest/".format(DOWNLOADDIR,BUCKET))

### (b) Make the full size repeatmasker.txt file (use this instead of `hg38_repeatmasker_clean.txt`)

In [79]:
#download repeatmasker file
!curl http://www.repeatmasker.org/genomes/hg38/RepeatMasker-rm405-db20140131/hg38.fa.out.gz \
    -o {DOWNLOADDIR}/hg38.fa.out.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  175M  100  175M    0     0  6971k      0  0:00:25  0:00:25 --:--:-- 7898k


In [81]:
%%bash -s "$DOWNLOADDIR"
DOWNLOADDIR=${1}
cd ${DOWNLOADDIR}
gunzip hg38.fa.out.gz
mv hg38.fa.out hg38_repeatmasker.txt

In [ ]:
print("Run at terminal (already done):")
print("gsutil cp \
{}/hg38_repeatmasker.txt \
{}/resourcestest/".format(DOWNLOADDIR,BUCKET))

# Step 2) Run the setup for RepEnrich2

## Download hg38 bowtie2 indexed files from biowulf and push to google cloud
found in `/fdb/igenomes/Homo_sapiens/UCSC/hg38/Sequence/Bowtie2Index/`

In [ ]:
print("run in terminal once files are downloaded locally (already done):\n")

print("gsutil cp \
{}/genome.1.bt2 \
{}/resourcestest/hg38_bowtie2/".format(DOWNLOADDIR,BUCKET))
print("gsutil cp \
{}/genome.2.bt2 \
{}/resourcestest/hg38_bowtie2/".format(DOWNLOADDIR,BUCKET))
print("gsutil cp \
{}/genome.3.bt2 \
{}/resourcestest/hg38_bowtie2/".format(DOWNLOADDIR,BUCKET))
print("gsutil cp \
{}/genome.4.bt2 \
{}/resourcestest/hg38_bowtie2/".format(DOWNLOADDIR,BUCKET))
print("gsutil cp \
{}/genome.fa \
{}/resourcestest/hg38_bowtie2/".format(DOWNLOADDIR,BUCKET))
print("gsutil cp \
{}/genome.rev.1.bt2 \
{}/resourcestest/hg38_bowtie2/".format(DOWNLOADDIR,BUCKET))
print("gsutil cp \
{}/genome.rev.2.bt2 \
{}/resourcestest/hg38_bowtie2/".format(DOWNLOADDIR))

## Build the annotations using the setup script (already done)

### copy `hg38_repeatmasker.txt` (or `hg38_repeatmasker_clean.txt`) and `RepEnrich2_setup.py` to biowulf then create a script to run the python script (so you can load the modules and submit as a sbatch job)

In [30]:
BIOWULF_FOLDER = ''
#ANNOTATION_FILE should be 'hg38_repeatmasker.txt' or 'hg38_repeatmasker_clean.txt'
ANNOTATION_FILE = 'hg38_repeatmasker.txt'

In [ ]:
print("create a script with this code:")
print("#!/bin/bash\n\
module load python\n\
module load bowtie\n\
module load bedtools\n\
module load samtools\n\
python RepEnrich2_setup.py {}/{}\
 /fdb/igenomes/Homo_sapiens/UCSC/hg38/Sequence/Bowtie2Index/genome.fa\
 {}/setup_folder_hg38".format(BIOWULF_FOLDER, ANNOTATION_FILE, BIOWULF_FOLDER))

#### run the script (takes several hours with the full genome.fa)

In [11]:
print("sbatch --mem=100g --cpus-per-task=10 --mail-type=ALL --time=24:00:00 setup_run.sh")

sbatch --mem=100g --cpus-per-task=10 --mail-type=ALL --time=24:00:00 setup_run.sh


### Compress the setup folder and push to google cloud
run all this in biowulf

In [2]:
print("compress the folder:")
print("tar czf setup_folder_hg38.tar.gz setup_folder_hg38")



compress the folder:
tar czf setup_folder_hg38.tar.gz setup_folder_hg38


In [ ]:
print("log into google cloud via biowulf:")
print("module load google-cloud-sdk")
print("gcloud auth login")

print("")

print("follow login instructions and then run command to push archived file to google cloud:")

print("gsutil cp \
setup_folder_hg38.tar.gz \
{}/resourcestest/".format(BUCKET))

# Step 3 + 4) Map the data to the genome and Run RepEnrich2 on the data
## start by building and pushing the image (already done)
docker must be installed locally (https://docs.docker.com/docker-for-mac/install/)

In [ ]:
print("run this locally at terminal in folder of docker image:")
print("docker build -t test-image .")

print("\ntag the image to point it to the google cloud project:")
print("docker tag test-image:latest us.gcr.io/{}/test-image".format(PROJECT_ID))

print("\npush the image to google cloud:")
print("docker push us.gcr.io/{}/test-image".format(PROJECT_ID))

print("\ncheck 'Container Registry' in gcc to find the new image")

#### once pushed find the container on google cloud and copy its full path
something like:

```us.gcr.io/BUCKET/test-image@sha256:04bc2af3cccd8618e6eafadc7d46e7fb24b2dc89e0e62ea0bdb26865d081f632```

paste the full path name into the input.json file wherever a docker variable is set

like

```"RepEnrichWorkflow.SamToBam.dockerimg": "us.gcr.io/PROJECT/test-image@sha256:04bc2af3cccd8618e6eafadc7d46e7fb24b2dc89e0e62ea0bdb26865d081f632"```

## Run the wdl
#### run this locally in terminal

In [44]:
FINAL_GC_OUTPUT='{}/Repenrich2_output'.format(GCOUTPUT)
COHORT = ''
SAMPLE=''

In [ ]:
print("gcloud alpha genomics pipelines run --project {} \
--pipeline-file {}/tools/wdl_pipeline.yaml --zones us-central1-f --memory 60 \
--logging {}/logs --inputs-from-file WDL={}/repEnrich2.wdl \
--inputs-from-file WORKFLOW_INPUTS={}/input.json \
--inputs-from-file WORKFLOW_OPTIONS={}/generic.google-papi.options.json \
--inputs WORKSPACE={}/Repenrich2_workspace \
--inputs OUTPUTS={} \
--labels=pipe=repenrich2,sample={},cohort={},user={}".format(PROJECT_ID, BUCKET, GCOUTPUT, WRKDIR, WRKDIR, WRKDIR, GCOUTPUT, FINAL_GC_OUTPUT, SAMPLE, COHORT, USER))

## Monitor the jobs (WIP, unfinished)

In [51]:
%%bash -s "$USER" "$COHORT" "$SAMPLE" "$PROJECT_ID"

USER=${1}
COHORT=${2}
SAMPLE=${3}
PROJECT_ID=${4}

PIPELABEL=repenrich2

gcloud compute instances list --project ${PROJECT_ID} --filter "labels.pipe=${PIPELABEL}" 

Listed 0 items.


In [ ]:
print('gcloud compute instances list --project {} --filter "labels.pipe={} labels.cohort={} labels.user={} labels.sample={}" '.format(PROJECT_ID,"repenrich2",COHORT, USER, SAMPLE))

In [54]:
%%bash

OPID=EIOn4NLdLRiDoInU-Myag_UBINn1oK--DSoPcHJvZHVjdGlvblF1ZXVl
gcloud alpha genomics operations describe ${OPID} \
--format='yaml(done, error, metadata.events)'

done: true
metadata:
  events:
  - description: start
    startTime: '2019-10-17T15:31:31.756157608Z'
  - description: pulling-image
    startTime: '2019-10-17T15:31:31.756227034Z'
  - description: localizing-files
    startTime: '2019-10-17T15:31:55.304646554Z'
  - description: running-docker
    startTime: '2019-10-17T15:31:55.304672508Z'
  - description: delocalizing-files
    startTime: '2019-10-19T02:23:47.909198410Z'
  - description: ok
    startTime: '2019-10-19T02:23:48.782883095Z'


In [56]:
%%bash


gcloud compute instances list

Listed 0 items.


## Download the output locally

In [ ]:
print("gsutil -mq cp {}/* {}/output".format(FINAL_GC_OUTPUT, WRKDIR))